In [1]:
import unidecode  
import json


### chargement des fichiers

In [4]:
###fichier secteur scolaire poitier
with open('Grand_poitier_Carte_scolaire.json') as json_data:
    data_dict = json.load(json_data)
    


In [5]:
data_dict

{'type': 'FeatureCollection',
 'name': 'Geoportail_WMS_Preview:Carte_scolaire',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'gml_id': 'F4932__976',
    'OBJECTID': 976,
    'NOM_SECTEUR_SCOLAIRE': 'PABLO NERUDA / MICROMEGAS',
    'RATTACHEMENT_SECTEUR_SCOLAIRE': 'Ecole mat. et Ã©lÃ©m. MicromÃ©gas et Ã©cole mat. et Ã©lÃ©m. P. NÃ©ruda',
    'Identifiant_bÃ¢timent': 'SCO_0016',
    'SHAPE.AREA': 1646876.97777606,
    'SHAPE.LEN': 6771.182041930051},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[0.390725118610502, 46.59576060473237],
       [0.390657452179044, 46.59643795044017],
       [0.389977541112438, 46.596230567397505],
       [0.389967999884216, 46.59622761448557],
       [0.389434154490492, 46.596069173779],
       [0.38941354647381, 46.596062467689556],
       [0.389287866491959, 46.59602158578867],
       [0.389213210775912, 46.59599554399892],
       [0.388951570724347,

In [6]:
#fichier école maternelle france
with open('maternelle_france.json') as json_data:
    data = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'maternelle_france.json'

In [7]:
#fichier elementaire france
with open('Primaire_france.json') as json_data:
    dataa = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'Primaire_france.json'

### recupération de la localisation des ecoles maternelle

In [5]:
mater={}
materpb={}

j=0
for i in range(0,14023):
    if 'POITIER' in data['features'][i]['properties']['localite_acheminement_uai']:
        
        materpb[j]={'nom':unidecode.unidecode(data['features'][i]['properties']['appellation_officielle'].lower()),'localisation':data['features'][i]['geometry']['coordinates']}
        mater[j]={'nom':data['features'][i]['properties']['appellation_officielle'],'localisation':data['features'][i]['geometry']['coordinates']}
        j+=1
        

### recupération de la localisation des ecoles elementares


In [6]:
elembc={}
elemP={}
j=0
for i in range(0,len(dataa)):
    if 'POITIER' in dataa[i]['fields']['localite_acheminement_uai']:
        elembc[j]={'nom':unidecode.unidecode(dataa[i]['fields']['appellation_officielle'].lower())}
        elemP[j]={'nom':dataa[i]['fields']['appellation_officielle'],'localisation':dataa[i]['fields']['position']}
        j+=1

### Fonction pour reécrire un fichier json

In [13]:

import json

def write_json(data,filename='Poitier_Final.geojson'):
    with open(filename,'w')as f:
        json.dump(data,f,indent=4)

ecoletst=[]
ecoletst.extend(data_dict['features'][0]['properties']['NOM_SECTEUR_SCOLAIRE'].split('/'))
ecoletst[0].strip()

### ajout de la localisation des écoles

In [15]:
for i in range (0,len(data_dict['features'])):
    data_dict['features'][i]['properties']['etablissement']={}
    h=0
    
    if '/' in data_dict['features'][i]['properties']['NOM_SECTEUR_SCOLAIRE']:
        ecole=[]
        ecole.extend(data_dict['features'][i]['properties']['NOM_SECTEUR_SCOLAIRE'].lower().split('/'))
        
        for elem in ecole:      
            #ecole maternel
            for j in range (0,len(mater)):
                mater[j]['localisation'].sort(reverse=True)
                if elem.strip() in materpb[j]['nom']:                
                    data_dict['features'][i]['properties']['etablissement'][h]=mater[j]                
                    h+=1
                    break
            #ecole elementaire            
            for f in range(0,len(elembc)):
                
                if elem.strip() =='saint exupery':
                    data_dict['features'][i]['properties']['etablissement'][h]= {'nom': "Ecole élémentaire d'application - St Exupery",'localisation': [46.5673307316241, 0.3679406527673295]}
                    h+=1
                    break 
                elif elem.strip() in elembc[f]['nom']:                   
                    data_dict['features'][i]['properties']['etablissement'][h]=elemP[f]  
                    h+=1
                    break   
    else:
        
        ecole=unidecode.unidecode(data_dict['features'][i]['properties']['NOM_SECTEUR_SCOLAIRE'].lower())
        
               
        for j in range (0,len(mater)):
           #print(ecole.strip(),'\n',materpb[j]['nom'],'\n')
            #mater[j]['localisation'].sort(reverse=True)
            
            if ecole.strip() in materpb[j]['nom']:
                data_dict['features'][i]['properties']['etablissement'][h]=mater[j]
                h+=1
                break
                
        for f in range(0,len(elembc)):
            

            if ecole.strip() =='valin - rey':
                data_dict['features'][i]['properties']['etablissement'][h]= {'nom': 'Ecole élémentaire Josué Valin La Rochelle','localisation': [46.15752612940727, -1.1483906114333056]}
           
            elif ecole.strip() in elembc[f]['nom']:
                data_dict['features'][i]['properties']['etablissement'][h]=elemP[f]
                h+=1
                break

                

write_json(data_dict)          

### gestion des ecoles non présente dans le fichier


In [10]:
u=[1,2,3,4,5,8,10,12,13]
for i in u:
    data_dict['features'][i]['properties']['etablissement']={}
    h=0
    if i==1:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom': 'Ecole maternelle La Citadelle','localisation': [43.501623582574936, -1.4684350656161123]}
    if i==2:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom':'ECOLE PRIMAIRE PUBLIQUE GRAND-BAYONNE','localisation':[43.492484, -1.477513]}
    if i==3:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom':'École Marie Curie','localisation':[43.507355, -1.486533]}
    if i==5:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom':'Ecole primaire petit bayonne','localisation':[43.491629, -1.472518] }
    
    if i==8:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom':'Groupe Scolaire Charles Malégarie','localisation':[43.502571, -1.451745]}
  
    if i==10:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom':'ECOLE MATERNELLE LAHUBIAGUE','localisation':[43.482588, -1.486097]}
    if i==12:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom': 'Ecole maternelle J-P Brana','localisation': [43.49779053883733, -1.4524009831994813]}
    
    
    
    if i ==13:
        data_dict['features'][i]['properties']['etablissement'][h]={'nom': 'Ecole du Prissé','localisation': [43.481264, -1.457703]}
        
write_json(data_dict)         

NameError: name 'write_json' is not defined

###  verification

In [9]:
for j in range (0,len(data_dict['features'])):
    print(j)
    print(data_dict['features'][j]['properties']['etablissement'])

0
{0: {'nom': 'Ecole maternelle Pablo Néruda', 'localisation': [46.589895943736614, 0.3796600783603964]}, 1: {'nom': 'Ecole Elémentaire Pablo Néruda', 'localisation': [46.589895943736614, 0.3796600783603964]}, 2: {'nom': 'Ecole maternelle Micromégas', 'localisation': [46.58552843653637, 0.37900009222943326]}, 3: {'nom': 'Ecole primaire Micromégas', 'localisation': [46.58597610494868, 0.3781696631510073]}}
1
{0: {'nom': 'Ecole maternelle Georges Brassens', 'localisation': [46.57211236260714, 0.3897902503911271]}, 1: {'nom': 'Ecole primaire Georges Brassens', 'localisation': [46.57173449077291, 0.3900500510494846]}}
2
{0: {'nom': 'Ecole maternelle Alphonse Daudet', 'localisation': [46.592370243828086, 0.36417057499214006]}, 1: {'nom': 'Ecole élémentaire Alphonse Daudet', 'localisation': [46.59201570371729, 0.36508950095593323]}}
3
{0: {'nom': 'Ecole maternelle Marcel Pagnol', 'localisation': [46.56666500661436, 0.3124697822667325]}, 1: {'nom': 'Ecole primaire Marcel Pagnol', 'localisatio

list(nom des ecoles fichier de base)

si nom ecole == list nom des ecoles
    rajoute long/latitude

In [37]:
maternel={}
elem={}
for i in range(0,len(localisation)):
    maternel[i]={'nom':ecolehavremat[i],'localisation':localisation[i]}
    elem[i]={'nom':ecolehavreElem[i],'localisation':localisationElem[i]} 

IndexError: list index out of range

### méthode laborieuse

In [ ]:
for i in range(0,len(data_dict['features'])):
   
    if '01' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Joseph Lakanal','type':'elementaire','localisation': [43.602306, 1.438747]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Alexandre Fourtanier','type':'Primaire','localisation': [43.602953, 1.449939]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Sermet','type':'maternelle','localisation': [43.607151, 1.441978]}

  
    elif '02' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Fabre','type':'elementaire','localisation': [43.595764, 1.443642]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Alexandre Fourtanier','type':'Primaire','localisation': [43.602953, 1.449939]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Port Garaud','type':'maternelle','localisation': [43.593580, 1.442788]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'toulouse maternelle Ricardie','type':'maternelle','localisation': [43.582710, 1.449342]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Ricardie 2','type':'elementaire','localisation': [43.582634, 1.448745]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'François Ricardie 3','type':'maternelle','localisation': [43.581355, 1.449824]}
        data_dict['features'][i]['properties']['etablissement7']={'nom_ecole': 'Pierre Dupont','type':'elementaire','localisation': [43.589127, 1.445518]}
        data_dict['features'][i]['properties']['etablissement8']={'nom_ecole': 'Calas ','type':'maternelle','localisation': [43.587137, 1.444246]}
        data_dict['features'][i]['properties']['etablissement9']={'nom_ecole': 'Jean Jaurès','type':'maternelle','localisation': [43.588576, 1.451786]}
        data_dict['features'][i]['properties']['etablissement10']={'nom_ecole': 'Jean Jaurès','type':'maternelle','localisation': [43.589008, 1.451420]}

      
    elif  '03' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Sarrat','type':'maternelle','localisation': [43.599392, 1.455124]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'calvinhac','type':'elementaire','localisation': [43.597986, 1.454873]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Michelet','type':'elemntaire','localisation': [43.603083, 1.455618]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Monge','type':'elementaire','localisation': [43.609017, 1.457756]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Reille','type':'maternelle','localisation': [43.609833, 1.459421]}

  
    elif '04' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Merly','type':'maternelle','localisation': [43.609468, 1.442680]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Nord','type':'elementaire','localisation': [43.610833, 1.441107]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Bayard-Matabiau','type':'maternelle','localisation': [43.612257, 1.448940]}
      
        

    elif '05' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Alexandre Falguière','type':'elementaire','localisation': [43.607389, 1.429829]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Pond Jumeaux','type':'elementaire','localisation': [43.615121, 1.422241]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Les Pond Jumeaux','type':'primaire','localisation': [43.614723, 1.422272]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Le Béarnais','type':'maternelle','localisation': [43.613607, 1.426280]}
        
 
    elif '06' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Amidonniers','type':'primaire','localisation': [43.609488, 1.420715]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Sept Deiers','type':'maternelle','localisation': [43.616625, 1.409462]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Petit Gragnague','type':'maternelle','localisation': [43.612437, 1.415192]}
     

    elif '07' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Musset','type':'elementaire','localisation': [43.622666, 1.430797]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Claude Nougaro','type':'elementaire','localisation': [43.620462, 1.434235]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Négreneys','type':'maternelle','localisation': [43.621408, 1.438653]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': ' Marie et Pierre Curie','type':'maternelle','localisation': [443.620750, 1.444077]}
        
     
    elif '08' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': ' Lucie Aubrac','type':'elementaire','localisation': [43.629621, 1.434894]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jules Ferry','type':'elementaire','localisation': [43.632912, 1.432481]}
        
    
    elif '09' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Jean Dieuzaide','type':'maternelle','localisation': [43.644940, 1.419755]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jean Monnet','type':'maternelle','localisation': [43.645920, 1.431761]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Lalande','type':'elementaire','localisation': [43.647967, 1.432226]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'De Gouges','type':'maternelle','localisation': [43.654362, 1.439618]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Jean Alphand','type':'elementaire','localisation': [43.655253, 1.434020]}

            
    elif '10' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Ernest Renan','type':'elementaire','localisation': [43.635811, 1.441799]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Ernest Renan','type':'groupe scolaire','localisation': [43.620462, 1.434235]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Niboul','type':'elementaire','localisation': [43.639931, 1.449984]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Borderouge','type':'maternelle et elementaire','localisation': [43.641270, 1.455089]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Jean Zay','type':'maternelle ','localisation':[43.641378, 1.447342]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Grand Selve','type':'elementaire ','localisation':[43.649669, 1.454251]}


    elif '11' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Maourine','type':'maternelle','localisation': [43.629804, 1.448863]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Lapujade','type':'elementaire','localisation': [43.628138, 1.461077]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Cuvier','type':'maternelle','localisation': [43.637221, 1.463011]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Cuvier','type':'elementaire','localisation': [43.636739, 1.463965]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': ' Michoun A','type':'maternelle','localisation': [43.626833, 1.467662]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': ' Michoun B','type':'maternelle','localisation': [43.629718, 1.467457]}
        data_dict['features'][i]['properties']['etablissement7']={'nom_ecole': ' Michoun ','type':'elementaire','localisation': [43.626841, 1.467715]}
                                                                                                                                        
            
    elif '12' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': ' Bonnefoy','type':'elementaire','localisation': [43.616721, 1.454204]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': ' Bonnefoy','type':'maternelle','localisation': [43.613606, 1.468714]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Marengo','type':'maternelle','localisation': [43.616747, 1.460919]}
        
        
    elif '13' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Jolimont','type':'elementaire','localisation': [43.613378, 1.467637]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jolimont','type':'maternelle','localisation': [43.616402, 1.453806]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Juncasse','type':'elementaire','localisation': [43.617579, 1.471292]}
  
    elif '14' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Chateaux de l\'hers','type':'elementaire','localisation': [43.600598, 1.484946]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jean Macé','type':'elementaire','localisation': [43.595627, 1.478314]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Terrasse','type':'primaire','localisation': [43.584147, 1.484949]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Terrasse','type':'maternelle','localisation': [43.583517, 1.485966]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Françoise Dolto','type':'maternelle','localisation': [43.586656, 1.479510]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Françoise Dolto','type':'primaire','localisation': [43.518256, 1.505323]}
                
                
               
    elif '15' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Armand Leygue','type':'maternelle','localisation': [43.596992, 1.464634]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jean Chaubet','type':'elementaire','localisation': [43.600540, 1.467594]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'La Gloire','type':'maternelle','localisation': [43.607692, 1.468824]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Bonhoure','type':'elemetaire','localisation': [43.602539, 1.466132]}
    
            
            
       
    elif '16' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Anatole France bd de la mediterranee','type':'primaire','localisation': [43.586303, 1.462803]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Anatole France','type':'elementaire','localisation': [43.585207, 1.462116]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Courrege','type':'elementaire','localisation': [43.583096, 1.476216]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Henri Guillaumet','type':'elementaire','localisation': [43.577063, 1.488593]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Montaudran','type':'maternelle','localisation': [43.574476, 1.484286]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Ecoles_St_O'}
     
    
   
    elif '17' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Pech David','type':'maternelle','localisation': [43.570815, 1.448665]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jules Julien','type':'elementaire','localisation': [43.573829, 1.453957]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Les Amies de la Maternelle Jules Julie','type':'maternelle','localisation': [43.574036, 1.453817]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Rangueil','type':'elementaire','localisation': [43.575209, 1.464085]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Sauzelong','type':'elementaire','localisation': [43.580534, 1.462353]}

      
    elif '18' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Lagrange','type':'elementaire','localisation': [43.577159, 1.437349]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Andre Daste','type':'elementaire','localisation': [43.580631, 1.438330]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Jean Moulin ','type':'maternelle','localisation': [43.582168, 1.441381]}   
 
    elif '19' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Le Pastel','type':'elementaire','localisation': [43.544967, 1.452888]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Jean-Pierre Vernant','type':'primaire','localisation': [43.550358, 1.457866]}
    
    
    
    elif '20' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Les Oustalous','type':'elementaire','localisation': [43.574649, 1.424630]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Maurice Jaquier','type':'elementaire','localisation': [43.575195, 1.418278]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Saint Stanislas','type':'primaire','localisation': [43.594412, 1.446719]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Sylvain Dauriac','type':'elementaire','localisation': [43.577912, 1.413973]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Papus','type':'elementaire','localisation': [43.575598, 1.412956]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Papus','type':'maternelle','localisation': [43.575122, 1.413979]}  
        data_dict['features'][i]['properties']['etablissement7']={'nom_ecole': 'Tabar','type':'maternelle','localisation': [43.570641, 1.413766]}  
    
    
    elif '21' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Maurice Becanne','type':'elementaire','localisation': [43.582668, 1.426634]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Annexe','type':'maternelle','localisation': [43.589478, 1.427392]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Benezet','type':'elementaire','localisation': [43.588393, 1.427863]}
  


    elif '22' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Alphonse de Lamartine','type':'maternelle','localisation': [43.595934, 1.434282]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Armand Duportal','type':'maternelle','localisation': [43.598996, 1.431823]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Lespinasse','type':'elementaire','localisation': [43.597512, 1.434558]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Molière','type':'maternelle','localisation': [43.593607, 1.431676]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Patte D\'oie','type':'elementaire','localisation': [43.595679, 1.425719]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Patte D\'oie','type':'maternelle','localisation': [43.594769, 1.426066]}  
        data_dict['features'][i]['properties']['etablissement7']={'nom_ecole': 'Polygone','type':'maternelle','localisation': [43.597959, 1.416543]}  
        data_dict['features'][i]['properties']['etablissement8']={'nom_ecole': 'Peire Godolin','type':'elementaire','localisation': [43.597840, 1.416346]}
        data_dict['features'][i]['properties']['etablissement9']={'nom_ecole': 'Fontaine Casselardit','type':'maternelle','localisation': [43.604254, 1.409380]}
        data_dict['features'][i]['properties']['etablissement10']={'nom_ecole': 'Fontaine Bayonne','type':'elemntaire','localisation': [43.600599, 1.416437]}  
        data_dict['features'][i]['properties']['etablissement11']={'nom_ecole': 'Les Gais Pinsons','type':'elementaire','localisation': [43.603713, 1.409045]}  
        data_dict['features'][i]['properties']['etablissement12']={'nom_ecole': 'Marcel Pagnol','type':'elementaire','localisation': [43.596800, 1.401156]}
        data_dict['features'][i]['properties']['etablissement13']={'nom_ecole': 'Marcel Pagnol','type':'maternelle','localisation': [43.595953, 1.401858]} 

        
      
    elif '23' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Clément Falcucci','type':'maternelle','localisation': [43.579674, 1.408289]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Georges Hyon','type':'elementaire','localisation': [43.579174, 1.409793]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Pierre de Ronsard','type':'elementaire','localisation': [43.584494, 1.407288]}
     
    
    
        
    elif '24' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Château d\'Anvcely','type':'maternelle','localisation': [43.618083, 1.398591]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Emile Littré','type':'maternelle','localisation': [43.607652, 1.377809]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Littré','type':'élémentaire','localisation': [43.608323, 1.377916]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Fleurance','type':'maternelle','localisation': [43.601057, 1.369915]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Fleurance','type':'elementaire','localisation': [43.598987, 1.370311]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Fleurance','type':'primaire','localisation': [43.598557, 1.370360]}
     
    
    
    
    elif '25' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Lardenne','type':'elementaire','localisation': [43.588440, 1.377863]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Gaston Dupouy','type':'elementaire','localisation': [43.580985, 1.380795]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Eugène Viollet-le-Duc','type':'élémentaire','localisation': [43.581183, 1.392048]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Ferdinand de Lesseps','type':'maternelle','localisation': [43.578746, 1.384276]}
        data_dict['features'][i]['properties']['etablissement5']={'nom_ecole': 'Ferdinand de Lesseps','type':'elementaire','localisation': [43.578999, 1.385232]}
        data_dict['features'][i]['properties']['etablissement6']={'nom_ecole': 'Léonce Bourliaguet','type':'maternelle','localisation': [43.577323, 1.395927]}
        data_dict['features'][i]['properties']['etablissement7']={'nom_ecole': 'Les Vergers','type':'elementaire','localisation': [43.581194, 1.398015]}
        data_dict['features'][i]['properties']['etablissement8']={'nom_ecole': 'Léonce Bourliaguet','type':'maternelle','localisation': [43.577323, 1.395927]}
     
    
    
    elif '26' in data_dict['features'][i]['properties']['numero_bassin_scolaire'] :
        data_dict['features'][i]['properties']['etablissement1']={'nom_ecole': 'Elsa Triolet','type':'elementaire','localisation': [43.572067, 1.394750]}
        data_dict['features'][i]['properties']['etablissement2']={'nom_ecole': 'Didier Daurat','type':'maternelle','localisation': [43.573762, 1.402538]}
        data_dict['features'][i]['properties']['etablissement3']={'nom_ecole': 'Daniel Faucher','type':'maternelle','localisation': [43.572548, 1.404773]}
        data_dict['features'][i]['properties']['etablissement4']={'nom_ecole': 'Buffon','type':'elementaire','localisation': [43.564383, 1.412240]}
        
write_json(data_dict)